In [4]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/sentiment/IMDB Dataset.csv').values
data

array([["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due t

In [17]:
import nltk
import numpy as np

In [5]:
X=data[:,0]
Y=data[:,1]

In [6]:
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
Y=l.fit_transform(Y)

In [7]:
Y

array([1, 1, 1, ..., 0, 0, 0])

In [8]:
def lower(X):
  reviews=[]
  m=X.shape[0]
  for i in range(m):
    review=X[i].lower()
    review=review.replace('<br /><br />',' ')
    reviews.append(review)
  return np.array(reviews)

In [9]:
def punctuate(X):
  reviews=[]
  m=X.shape[0]
  for i in range(m):
    review=X[i]
    for ele in review:
      if ele in puncts:
        review=review.replace(ele,'')
    reviews.append(review)
  return np.array(reviews)

In [10]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
tokenize=RegexpTokenizer(r'\w+')
ss=SnowballStemmer('english')
en_stopwords=set(stopwords.words('english'))

In [13]:
def tokenizer(X):
  reviews=[]
  m=X.shape[0]
  for i in range(m):
    review=X[i]
    tokens=tokenize.tokenize(review)
    review=' '.join(tokens)
    reviews.append(review)
  return np.array(reviews)

In [15]:
def stopword_rm(X):
  reviews=[]
  m=X.shape[0]
  for i in range(m):
    review=X[i]
    tokens=tokenize.tokenize(review)
    new_tokens=[token for token in tokens if token not in en_stopwords]
    new_tokens=' '.join(new_tokens)
    reviews.append(new_tokens)
  return np.array(reviews)

In [14]:
def stemming(X):
  reviews=[]
  m=X.shape[0]
  for i in range(m):
    review=X[i]
    tokens=tokenize.tokenize(review)
    stem=[ss.stem(t) for t in tokens]
    stem=' '.join(stem)
    reviews.append(stem)
  return np.array(reviews)

In [18]:
X=lower(X)

In [20]:
def mainfunction(X):
  reviews=[]
  m=X.shape[0]
  for i in range(m):
    review=X[i]
    review=review.lower()
    review=review.replace('<br /><br />',' ')
    #for ele in review:
      #if ele in puncts:
        #review=review.replace(ele,'')
    tokens=tokenize.tokenize(review)
    stem=[ss.stem(t) for t in tokens]
    new_tokens=[token for token in tokens if token not in en_stopwords]
    new_tokens=' '.join(new_tokens)
    reviews.append(new_tokens)
  return np.array(reviews)

In [21]:
X1=mainfunction(X)

In [22]:
from sklearn.model_selection import train_test_split
x1_train,x1_test,y1_train,y1_test=train_test_split(X1,Y,test_size=0.4)
x1_val=x1_test[:10000]
y1_val=y1_test[:10000]
x1_test=x1_test[10000:]
y1_test=y1_test[10000:]

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x1cv=cv.fit_transform(x1_train)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
x1tf=tfidf.fit_transform(x1_train)

In [25]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()

In [26]:
mnb.fit(x1cv,y1_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
mnb.score(x1cv,y1_train)

0.9096333333333333

In [28]:
x1cv_val=cv.transform(x1_val)
x1cv_test=cv.transform(x1_test)

In [29]:
mnb.score(x1cv_test,y1_test)

0.8614

In [30]:
y_pred=mnb.predict(x1cv_test)

In [31]:
print(y1_test)

[1 0 0 ... 0 1 1]


In [32]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y1_test, y_pred, average='binary')

(0.8821324448146606, 0.8378164556962026, 0.859403530127815, None)

In [33]:
from sklearn.metrics import confusion_matrix
cnf=confusion_matrix(y1_test,y_pred)

In [34]:
print(cnf)

[[4378  566]
 [ 820 4236]]


In [35]:
mnb1=MultinomialNB()

In [36]:
mnb1.fit(x1tf,y1_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [37]:
mnb1.score(x1tf,y1_train)

0.9128

In [38]:
x1tf_test=tfidf.transform(x1_test)

In [39]:
mnb1.score(x1tf_test,y1_test)

0.8701

In [40]:
y_pred1=mnb1.predict(x1tf_test)

In [41]:
cnf1=confusion_matrix(y1_test,y_pred1)
print(cnf1)

[[4366  578]
 [ 721 4335]]


In [42]:
precision_recall_fscore_support(y1_test, y_pred1, average='binary')

(0.8823529411764706, 0.8573971518987342, 0.8696960577791153, None)

In [43]:
from sklearn.linear_model import LogisticRegression

In [44]:
params=[
        {'C':[0.01,0.03,0.1,0.3,1,3,10,30,100]}
]

In [45]:
from sklearn.model_selection import GridSearchCV

clf=LogisticRegression()
gs=GridSearchCV(estimator=clf,param_grid=params,scoring='f1',cv=5,n_jobs=-1)

In [46]:
x1tf_val=tfidf.transform(x1_val)

In [47]:
clf.fit(x1tf,y1_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
clf.score(x1tf,y1_train)

0.9359666666666666

In [49]:
gs.fit(x1tf_val,y1_val)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [50]:
gs.best_estimator_

LogisticRegression(C=3, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [51]:
clf=LogisticRegression(C=3,max_iter=20)

In [ ]:
x1tf

In [52]:
clf.fit(x1tf,y1_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=3, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=20,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
clf.score(x1tf,y1_train)

0.9410666666666667

In [54]:
clf.score(x1tf_val,y1_val)

0.8904

In [55]:
clf.score(x1tf_test,y1_test)

0.8963

In [56]:
clf.score(x1tf_val,y1_val)

0.8904

In [57]:
y_pred2=clf.predict(x1tf_test)

In [58]:
cnf2=confusion_matrix(y1_test,y_pred2)
print(cnf2)

[[4377  567]
 [ 470 4586]]


In [59]:
precision_recall_fscore_support(y1_test, y_pred2, average='binary')

(0.8899670095090239, 0.9070411392405063, 0.8984229601332158, None)